In [4]:
# Insurance Premium Calculator ver 3.2

# import
import pandas as pd
import numpy as np

# data 불러오기
insurance_premium=pd.read_excel('./insurance_premium.xlsx')
insurance_premium_info=pd.read_excel('./insurance_premium_info.xlsx')

# 보험료 계산기
# insurance_premium : data
# disease_risk_li : 입력된 질병별 위험도
# age : 나이
# insurance_premium_max : 이용자가 원하는 보험료
def calculator(insurance_premium, disease_risk_li, age, insurance_premium_max):
    
    # 나이대에 따른 최대 보험료, 최소 보험료 뽑아오기
    if 19<age<30:
        price_min = np.array(insurance_premium.iloc[:,2])
        price_max = np.array(insurance_premium.iloc[:,3])
        price_gap = np.floor(np.array(price_max-price_min))
        
    else :
        price_min = np.array(insurance_premium.iloc[:,4])
        price_max = np.array(insurance_premium.iloc[:,5])
        price_gap = np.floor(np.array(price_max-price_min))
        
    # 기본 특약 계산을 위한 변수
    disease_risk_avg = sum(disease_risk_li)/4
    
    # 특약별 보험료 계산을 위한 리스트
    value_min=[0,0,0,0,0]; value_rate=[0,0,0,0,0]; sum_value_rate=[0,0,0,0,0]
    
    # return을 위한 리스트들
    list_1=['기본특약','심혈관질환','대사질환','뇌혈관질환','간질환','총합']
    list_2=[0,0,0,0,0,0]; list_3=[0,0,0,0,0,0]; list_4=[0,0,0,0,0,0]
    
    # 보험료 계산에 필요한 값들 정리
    for i in range(5):
        value_min[i]=sum(price_min*np.array(insurance_premium.iloc[:,i+6]))
    sum_value_min=sum(value_min)
    
    value_rate[0]=price_gap*np.array(insurance_premium['기본특약'])*disease_risk_avg
    for i in range(1,5):
        value_rate[i]=price_gap*np.array(insurance_premium.iloc[:,i+6])*disease_risk_li[i-1]

    sum_value_rate_disease=np.sum(np.array(value_rate[1:5]))
    
    for i in range(5):
        list_4[i]=sum(price_max*np.array(insurance_premium.iloc[:,i+6]))
    list_4[5]=sum(list_4)
    
    # 보험료 출력을 위한 보험료 재조정과 출력
    if sum_value_min>insurance_premium_max:
        
        for i in range(5):
            list_3[i]=value_min[i]
        list_3[5]=sum(list_3)
        
        ipc_df=pd.DataFrame({'내보험료':list_2, '적정보험료':list_3, '최대보험료':list_4}, index=list_1)
        for i in range(0,3):
            ipc_df.iloc[:,i]=ipc_df.iloc[:,i].map('{:,.0f}'.format)+'원'
        
        return ipc_df

    elif sum_value_min+sum_value_rate_disease>insurance_premium_max:

        # 보험료 예산에 맞춘 보험료 재조정
        for i in range(1,5):
            sum_value_rate[i]=np.floor(sum(value_rate[i]*((insurance_premium_max-sum_value_min)/sum_value_rate_disease)))

        # 보험료 출력을 위한 list 저장
        list_2[0]=value_min[0]; list_3[0]=value_min[0]
        for i in range(1,5):
            list_2[i]=int(value_min[i]+sum_value_rate[i]); list_3[i]=value_min[i]+int(sum(value_rate[i]))
        list_2[5]=sum(list_2); list_3[5]=sum(list_3)
        
        ipc_df=pd.DataFrame({'내보험료':list_2, '적정보험료':list_3, '최대보험료':list_4}, index=list_1)
        for i in range(0,3):
            ipc_df.iloc[:,i]=ipc_df.iloc[:,i].map('{:,.0f}'.format)+'원'
        
        return ipc_df
        
    else:

        # 보험료 예산에 맞춘 보험료 재조정
        tmp=sum_value_min+sum_value_rate_disease
        tmp_value1_rate=value_rate[0]*(insurance_premium_max-tmp)/sum(value_rate[0])

        # 보험료 출력을 위한 list 저장
        if value_min[0]+sum(tmp_value1_rate)>=value_min[0]+sum(value_rate[0]):
            list_2[0]=value_min[0]+sum(value_rate[0]); list_3[0]=value_min[0]+sum(value_rate[0])
        else:
            list_2[0]=value_min[0]+sum(tmp_value1_rate); list_3[0]=value_min[0]+sum(value_rate[0])
        for i in range(1,5):
            list_2[i]=value_min[i]+int(sum(value_rate[i])); list_3[i]=value_min[i]+int(sum(value_rate[i]))
        list_2[5]=sum(list_2); list_3[5]=sum(list_3)
            
        ipc_df=pd.DataFrame({'내보험료':list_2, '적정보험료':list_3, '최대보험료':list_4}, index=list_1)
        for i in range(0,3):
            ipc_df.iloc[:,i]=ipc_df.iloc[:,i].map('{:,.0f}'.format)+'원'
        
        return ipc_df

In [5]:
# import
import pandas as pd
import numpy as np

# data 불러오기 및 입력하기
insurance_premium=pd.read_excel('./insurance_premium.xlsx')
input_data = [0.5,0.7,0.2,0.2]
insurance_premium_max=80000
age=22


if 19<age<30 or 39<age<50:
    ipc_df=calculator(insurance_premium, input_data, age,insurance_premium_max)
    
    if ipc_df.iloc[5,0]=='0원':
        print('설정한 최소 보험료가 너무 낮습니다. 계산된 최소 보험료 : %s'%ipc_df.iloc[5,1])
        
    else:
        display(ipc_df)
    
else:
    print('죄송합니다. 현재 고객님을 위한 상품이 준비되어 있지 않습니다.')
    print('현재 저희는 20대와 40대를 위한 상품만 제공되고 있습니다.')

,내보험료,적정보험료,최대보험료
기본특약,"37,802원","37,802원","594,775원"
심혈관질환,"30,074원","46,719원","88,250원"
대사질환,649원,878원,"1,124원"
뇌혈관질환,"8,683원","13,484원","61,400원"
간질환,"2,791원","3,633원","12,037원"
총합,"79,999원","102,516원","757,586원"
